In [1]:
import os
import json
from iqcc_cloud_client import IQCC_Cloud

# Set your quantum computer backend
quantum_computer_backend = "gilboa"  # for example qc_qwfix
iqcc = IQCC_Cloud(quantum_computer_backend=quantum_computer_backend)
# Get the latest state and wiring files
latest_wiring = iqcc.state.get_latest("wiring")
latest_state = iqcc.state.get_latest("state")

# Get the state folder path from environment variable
quam_state_folder_path = os.environ["QUAM_STATE_PATH"]

# Save the files
with open(os.path.join(quam_state_folder_path, "wiring.json"), "w") as f:
    json.dump(latest_wiring.data, f, indent=4)

with open(os.path.join(quam_state_folder_path, "state.json"), "w") as f:
    json.dump(latest_state.data, f, indent=4)

2025-06-25 11:44:23,786 - qm - INFO     - Starting session: 0022ccd7-60da-436a-b6d8-ffec32430a95


In [2]:
from qm import QuantumMachinesManager, SimulationConfig
from qm_saas import QoPVersion, QmSaas
from quam_libs.components import (
    MeasureMacro,
    ResetMacro,
    VirtualZMacro,
    CZMacro,
    DelayMacro,
)
from qiskit_qm_provider import (
    FluxTunableTransmonBackend,
    QISKIT_PULSE_AVAILABLE
)
from quam.components.macro import PulseMacro
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector, QuantumRegister
import numpy as np
from quam_libs.components.quam_root import QuAM as IQCC_Quam
import warnings
import logging

# Silence DeprecationWarnings from qiskit.pulse
warnings.filterwarnings("ignore", category=DeprecationWarning, module=r"qiskit\.pulse")
# Filter all DeprecationWarnings
logging.getLogger("qiskit").setLevel(logging.ERROR)


In [8]:
file_path = "/pulse_level/qua/quam_state"
machine = IQCC_Quam.load()

In [9]:
if not machine.active_qubits[0].macros:
    from qiskit_qm_provider.backend.backend_utils import add_basic_macros_to_machine
    add_basic_macros_to_machine(machine)

In [10]:
backend = FluxTunableTransmonBackend(machine, qmm=iqcc)
print(backend.target)

/Users/arthurostrauss/Library/CloudStorage/OneDrive-QMMachinesLTD/GitHub/qiskit-qm-provider/qiskit_qm_provider/backend/flux_tunable_transmon_backend.py:58: UserWarning: qiskit.pulse is not available, channel mapping will not be set.
  warnings.warn("qiskit.pulse is not available, channel mapping will not be set.")


Target: Transmon based QuAM
Number of qubits: 4
Instructions:
	x
		(0,):
			Duration: 4.8e-08 sec.
		(1,):
			Duration: 4.8e-08 sec.
		(2,):
			Duration: 4.8e-08 sec.
		(3,):
			Duration: 4.8e-08 sec.
	rz
		(0,):
			Duration: 0 sec.
			Error Rate: 0
		(1,):
			Duration: 0 sec.
			Error Rate: 0
		(2,):
			Duration: 0 sec.
			Error Rate: 0
		(3,):
			Duration: 0 sec.
			Error Rate: 0
	sx
		(0,):
			Duration: 4.8e-08 sec.
		(1,):
			Duration: 4.8e-08 sec.
		(2,):
			Duration: 4.8e-08 sec.
		(3,):
			Duration: 4.8e-08 sec.
	delay
		(0,):
		(1,):
		(2,):
		(3,):
	reset
		(0,):
		(1,):
		(2,):
		(3,):
	measure
		(0,):
			Duration: 1.5e-06 sec.
		(1,):
			Duration: 1.5e-06 sec.
		(2,):
			Duration: 1.5e-06 sec.
		(3,):
			Duration: 1.5e-06 sec.
	cz
		(2, 1)
	if_else
	while_loop
	for_loop
	switch_case
	box



In [ ]:
from rl_qoc import (
    QEnvConfig,
    QiskitConfig,
    ExecutionConfig,
    BenchmarkConfig, GateTarget,
)
from rl_qoc.rewards import StateReward
from rl_qoc.qua import QMConfig, QMEnvironment

from gymnasium.spaces import Box
if QISKIT_PULSE_AVAILABLE:
    import qiskit.pulse as qp

def apply_parameters(qc: QuantumCircuit, params: ParameterVector, qreg: QuantumRegister, *args):
    idx_0 = qreg.index(qreg[0])
    if len(qreg) > 1:
        qc.rzz(params[0], qreg[0], qreg[1])
        if QISKIT_PULSE_AVAILABLE:
            idx_1 = qreg.index(qreg[1])
            with qp.build(backend) as sched:
                qp.play(qp.Constant(duration=120, amp=0.1), backend.flux_channel(idx_0))
                qp.play(qp.Constant(duration=120, amp=params[0]), backend.control_channel((idx_0, idx_1))[0])
            qc.add_calibration("rzz", (0, 1), sched, params.params if isinstance(params, ParameterVector) else params)

    else:
        qc.rx(params[0], qreg[0])
        if QISKIT_PULSE_AVAILABLE:
            with qp.build(backend) as sched:
                qp.play(
                    qp.Drag(duration=120, amp=params[0], sigma=40, beta=0.2),
                    backend.drive_channel(idx_0),
                )
            qc.add_calibration("rx",(0,), sched, params.params if isinstance(params, ParameterVector) else params)
    if QISKIT_PULSE_AVAILABLE:
        with qp.build(backend, name="x_custom") as sched:
            qp.play(
                qp.Drag(duration=120, amp=0.3, sigma=40, beta=0.2),
                backend.drive_channel(idx_0),
            )
        qc.add_calibration("x",(0,), sched)
        qc.add_calibration("x",(1,), sched)


physical_qubits = [0]
gate = "cz" if len(physical_qubits) > 1 else "x"
n_reps = [2, 5]
input_type = "INPUT_STREAM"
num_updates = 500

config = QEnvConfig(
    target=GateTarget(physical_qubits=physical_qubits, gate=gate),
    backend_config= QMConfig(
        backend=backend,
        parametrized_circuit=apply_parameters,
        input_type=input_type,
        num_updates=num_updates,
    ),
    execution_config=ExecutionConfig(
        batch_size=1,
        n_shots=100,
        n_reps=n_reps,
        control_flow_enabled=True,
    ),
    reward=StateReward("pauli4"),
    action_space=Box(low=-1, high=1, shape=(1,), dtype=np.float32),
)

q_env = QMEnvironment(config)

In [ ]:
q_env.rl_qoc_training_qua_prog()

In [ ]:
from qm import generate_qua_script

print(
    generate_qua_script(
        backend.quantum_circuit_to_qua(q_env.real_time_circuit).result_program.dsl_program
    )
)

In [ ]:
from qm import generate_qua_script

prog = q_env.rl_qoc_training_qua_prog()
print(generate_qua_script(prog))

In [ ]:
from pathlib import Path
import json
from qiskit_qm_provider import ParameterPool

path = Path.home() / "qm_saas_config.json"
with open(path, "r") as f:
    config = json.load(f)
email = config["email"]
password = config["password"]
# Initialize QOP simulator client
client = QmSaas(email=email, password=password, host="qm-saas.dev.quantum-machines.co")

# Choose your QOP version (QOP2.x.y or QOP3.x.y)
version = QoPVersion.latest
client.close_all()

In [ ]:
from qm import SimulationConfig, LoopbackInterface

config = machine.generate_config()
instance = client.simulator(version=version)
instance.spawn()
# Initialize QuantumMachinesManager with the simulation instance details
qmm = QuantumMachinesManager(
    host=instance.host,
    port=instance.port,
    connection_headers=instance.default_connection_headers,
)
simulation_config = SimulationConfig(
    int(10000),
    include_analog_waveforms=True,
    simulation_interface=LoopbackInterface([("con1", 1, "con1", 1), ("con1", 2, "con1", 2)]),
)
backend.qmm = qmm
backend.set_options(simulate=simulation_config)

In [ ]:
backend._qmm

In [ ]:
backend.qmm = iqcc

In [ ]:
from qiskit_experiments.library.randomized_benchmarking import StandardRB
from qiskit_qm_provider.primitives.qm_sampler import QMSamplerV2, QMSamplerOptions

sampler = QMSamplerV2(
    backend,
    options=QMSamplerOptions(
        input_type="INPUT_STREAM",
    ),
)
exp = StandardRB([0], [5, 30, 60, 90, 120], backend=backend, num_samples=5)
exp._transpiled_circuits()[0].draw("mpl")

In [ ]:
print(generate_qua_script(backend.get_run_program(100, exp._transpiled_circuits())))

In [ ]:
len(exp._transpiled_circuits())

In [ ]:
exp_data = exp.run(sampler=sampler, backend_run=True)

In [ ]:
from qiskit.circuit import QuantumCircuit, Parameter, Gate

physical_qubits = (1, 2)
qc = QuantumCircuit(2)
amp = Parameter("amp")
gate = Gate("x_cal", 1, [amp])
qc.append(gate, [0])
qc.append(gate, [1])
qc.draw("mpl")

In [ ]:
from qiskit_qm_provider import QMInstructionProperties
from qm.qua import *


def play_custom_x_1(amp):
    machine.active_qubits[physical_qubits[0]].xy.play("x180", amplitude_scale=amp)


def play_custom_x_2(amp):
    machine.active_qubits[physical_qubits[1]].xy.play("x180", amplitude_scale=amp)


instruction_prop1 = QMInstructionProperties(qua_pulse_macro=play_custom_x_1)
instruction_prop2 = QMInstructionProperties(qua_pulse_macro=play_custom_x_2)

backend.target.add_instruction(gate, {0: instruction_prop1, 1: instruction_prop2})